In [52]:
# import dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

In [53]:
# Read in the file
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


DATA PREPROCESSING

In [54]:
# look at what data types you are working with
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [55]:
# Remove cryptocurrencies that aren't trading
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace=True)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [56]:
# Remove all the cryptocurrencies that dont have an algorithm defined
crypto_df = crypto_df[crypto_df.Algorithm != '']

In [57]:
# Remove the IsTrading column
crypto_df.drop(columns=['IsTrading'], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [58]:
# (null values)
# Use isnull method to check for missing values
# Loop through each column, check if there are null values, sum them up, and print readable total

for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} nullvalues")

Column CoinName has 0 nullvalues
Column Algorithm has 0 nullvalues
Column ProofType has 0 nullvalues
Column TotalCoinsMined has 459 nullvalues
Column TotalCoinSupply has 0 nullvalues


In [59]:
# Remove all cryptocurrencies with at least one null value
crypto_df = crypto_df.dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [60]:
# Remove all cryptocurrencies without coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined']>0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [61]:
# Store the names of all cryptocurrencies on a DataFramed named coins_name
coins_name = pd.DataFrame([crypto_df.CoinName]).transpose()
coins_name.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [62]:
# use the crypto_df.index as the index for this new DataFrame.
# coins_name.set_index(crypto_df['Unnamed: 0'])

In [63]:
# Remove the CoinName column.
crypto_df.drop(columns=["CoinName"], inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [64]:
# Create dummies variables for all of the text features, and store the resulting data on a DataFrame named X.
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
# Standardize the data
X = StandardScaler().fit_transform(X)
X[:5]

array([[-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
        -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
        -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
        -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
        -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
        -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
        -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
        -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
        -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.39879994, -0.0433963 , -0.1

REDUCING DATA DIMENSIONS USING PCA

In [66]:
# reduce it to 3 dimensions
essentialEigenVectors = PCA(n_components=3)

In [67]:
essentialEigenVectors.fit(X)

PCA(n_components=3)

In [68]:
print(essentialEigenVectors.explained_variance_ratio_)

[0.02792059 0.02136095 0.02049865]


In [69]:
pcs = essentialEigenVectors.transform(X)

In [70]:
pcs_df = pd.DataFrame(pcs, index=crypto_df.index, columns=['pc1', 'pc2', 'pcs3'])
pcs_df.head()

,pc1,pc2,pcs3
42,-0.302826,0.900306,-0.550951
404,-0.286205,0.900530,-0.551477
1337,2.317847,1.585897,-0.631227
BTC,-0.156913,-1.284662,0.199533
ETH,-0.172724,-1.960039,0.374533


CLUSTERING CRYPTOCURRENCIES USING K-MEANS

In [88]:
# Elbow Curve
inertia = []
k = list(range(1, 11))
clusters = range(1,30)
# Looking for the best k
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [89]:
# Initialize K-means model
model = KMeans(n_clusters=10, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Add the predicted class columns
pcs_df["class"] = model.labels_
pcs_df.head()

,pc1,pc2,pcs3,class
42,-0.302826,0.900306,-0.550951,0
404,-0.286205,0.900530,-0.551477,0
1337,2.317847,1.585897,-0.631227,7
BTC,-0.156913,-1.284662,0.199533,3
ETH,-0.172724,-1.960039,0.374533,2


In [90]:
# Create a new DataFrame named “clustered_df"
clustered_df = crypto_df.join(pcs_df, how='inner')

In [91]:
clustered_df = clustered_df.join(coins_name, how='inner')
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,pc1,pc2,pcs3,class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.302826,0.900306,-0.550951,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.286205,0.900530,-0.551477,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.317847,1.585897,-0.631227,7,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.156913,-1.284662,0.199533,3,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.172724,-1.960039,0.374533,2,Ethereum


VISUALIZING RESULTS

In [92]:
from sklearn.cluster import AgglomerativeClustering
import plotly.figure_factory as ff

In [93]:
# 3D Scatter Plot

fig = px.scatter_3d(clustered_df, x="pc1", y="pc2", z="pcs3",
                    color='class', symbol='class', hover_name="CoinName", hover_data=["Algorithm"])

fig.update_layout(
    legend=dict(
        x=0,
        y=1))

fig.show()

In [100]:
# Use hvplot.table to create data table with all current tradable cryptocurrencies
clustered_df.hvplot.table(columns= ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class'])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [101]:
# Scatter Plot using hvplot.scatter

clustered_df.hvplot.scatter(x = 'TotalCoinsMined', y = 'TotalCoinSupply', hover_cols=['CoinName'], by ='class')

:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)